In [65]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(pheatmap)
library(RColorBrewer)
library(viridis)
source('/home/jupyter/BRI_Figures_Final_V1/helper_function/helper_function_IHA.r')

In [66]:
df_BR1 <- read.csv("/home//jupyter/BRI_Figures_Final_V1/Figure4/04_DEG_AgeGroup_CMVneg_vs_CMVpos/Deseq2_Result_BR1_CMVpos_vs_CMVneg.csv")
df_BR2 <- read.csv("/home//jupyter/BRI_Figures_Final_V1/Figure4/04_DEG_AgeGroup_CMVneg_vs_CMVpos/Deseq2_Result_BR2_CMVpos_vs_CMVneg.csv")


df_BR1_filtered <- df_BR1 %>% filter(abs(log2FoldChange) > 0.1, padj < 0.05,celltype=='KLRF1- GZMB+ CD27- EM CD8 T cell')
df_BR2_filtered <- df_BR2 %>% filter(abs(log2FoldChange) > 0.1, padj < 0.05,celltype=='KLRF1- GZMB+ CD27- EM CD8 T cell')


In [67]:
df_BR1_filtered<-df_BR1_filtered %>% 
  group_by(Direction) %>% 
  top_n(5, wt = abs(log2FoldChange)*-log10(padj))

In [68]:
df_BR2_filtered<-df_BR2_filtered %>% 
  group_by(Direction) %>% 
  top_n(5, wt = abs(log2FoldChange)*-log10(padj))

In [69]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA_meta_data-2024-05-09.csv")

In [70]:
meta_data_subset=meta_data %>% filter(sample.visitName=="Flu Year 1 Day 0")
file_list<-paste0("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data_subset$pbmc_sample_id,".csv")

df_list<-read_pseudobulk_expression(file_list)


[1] "Total reading time: 10.46 seconds"
[1] "The length of the list matches the length of the input path."


In [71]:
  
celltype_list <- lapply(df_list, function(df) { df <- df[unique(c(df_BR2_filtered$gene,c(df_BR1_filtered$gene))), grep('Adaptive NK cell', names(df), fixed = TRUE),drop=FALSE] })


In [72]:
exp_matrix<-do.call(cbind,celltype_list)

colnames(exp_matrix)<-sub(":.*", "", colnames(exp_matrix))
rownames(meta_data_subset)<-meta_data_subset$pbmc_sample_id

In [73]:
write.csv(exp_matrix,"exp_matrix_KLRFneg_GZMBpos_CD27neg_CD8EM.csv")

In [74]:
# Get unique values from both vectors
unique_values <- union(df_BR1_filtered$gene, df_BR2_filtered$gene)

# Create a vector to store overlap information
overlap_info <- sapply(unique_values, function(value) {
  if (value %in% df_BR1_filtered$gene && value %in% df_BR2_filtered$gene) {
    return("Overlap")
  } else if (value %in% df_BR1_filtered$gene) {
    return("Only in Young")
  } else {
    return("Only in Older")
  }
})


In [75]:
df <- data.frame(Unique_Values = unique_values, Overlap_Info = overlap_info, stringsAsFactors = FALSE)

In [76]:
write.csv(df,"gene_overlaps.csv")